# Pre-calculate Shortest Paths for Traffic Points\n\nThis notebook calculates and caches shortest paths between all 10 fixed traffic monitoring points.\n\n**Goal:** Save ~2 seconds per shortest path request by pre-computing and caching results.

In [ ]:
import json\nimport sys\nfrom pathlib import Path\nimport time\nfrom datetime import datetime\nimport networkx as nx\n\n# Add server path so we can use routing functions\nsys.path.insert(0, str(Path.cwd()))\n\nimport config as global_config\nfrom server import routing\n\nprint('✅ Imports successful')

In [ ]:
# Load traffic points\ntraffic_points_file = Path('collector/outputs/traffic_points_to_edges.json')\n\nwith open(traffic_points_file, 'r') as f:\n    traffic_data = json.load(f)\n\npoints = traffic_data['mapping']\nprint(f'📍 Loaded {len(points)} traffic monitoring points')\nfor p in points:\n    print(f'  - {p[\"name\"]}')

In [ ]:
# Load road network graph\nprint('🗺️  Loading road network graph...')\nstart = time.time()\n\nG = routing.load_graph()\n\nelapsed = time.time() - start\nprint(f'✅ Graph loaded in {elapsed:.2f}s')\nprint(f'   Nodes: {G.number_of_nodes():,}')\nprint(f'   Edges: {G.number_of_edges():,}')

In [ ]:
# Calculate all shortest paths\nprint('\\n🚀 Starting shortest path pre-calculation...')\nprint(f'   Total combinations: {len(points)} × {len(points)-1} = {len(points) * (len(points)-1)}\\n')\n\ncache = {}\nsuccess_count = 0\nfail_count = 0\ntotal_time = 0\n\n# Calculate for all origin-destination pairs\nfor i, origin_point in enumerate(points, 1):\n    for j, dest_point in enumerate(points, 1):\n        # Skip same point\n        if origin_point['name'] == dest_point['name']:\n            continue\n        \n        origin_lat = origin_point['lat']\n        origin_lon = origin_point['lon']\n        dest_lat = dest_point['lat']\n        dest_lon = dest_point['lon']\n        \n        route_label = f\"{origin_point['name']} → {dest_point['name']}\"\n        print(f'[{success_count + fail_count + 1}/{len(points) * (len(points)-1)}] {route_label}', end='... ')\n        \n        try:\n            start = time.time()\n            \n            # Find nearest nodes\n            origin_node = routing.find_nearest_node(origin_lat, origin_lon)\n            dest_node = routing.find_nearest_node(dest_lat, dest_lon)\n            \n            if origin_node is None or dest_node is None:\n                print('❌ Could not find nearest nodes')\n                fail_count += 1\n                continue\n            \n            # Calculate shortest path using length\n            path = nx.shortest_path(G, origin_node, dest_node, weight='length')\n            \n            # Get path details\n            edges = []\n            total_distance = 0\n            \n            for idx in range(len(path) - 1):\n                u, v = path[idx], path[idx + 1]\n                edge_data = G[u][v][0]  # Get first edge \n                length = edge_data.get('length', 0)\n                total_distance += length\n                edges.append([u, v, 0])  # Store as [u, v, key]\n            \n            elapsed = time.time() - start\n            total_time += elapsed\n            \n            # Create cache key\n            cache_key = f\"{origin_lat},{origin_lon}|{dest_lat},{dest_lon}\"\n            \n            # Store in cache\n            cache[cache_key] = {\n                'origin': {'name': origin_point['name'], 'lat': origin_lat, 'lon': origin_lon},\n                'destination': {'name': dest_point['name'], 'lat': dest_lat, 'lon': dest_lon},\n                'origin_node': origin_node,\n                'dest_node': dest_node,\n                'path': path,\n                'edges': edges,\n                'distance_m': total_distance,\n                'distance_km': round(total_distance / 1000, 2),\n                'calculated_at': datetime.now().isoformat()\n            }\n            \n            print(f'✅ {total_distance/1000:.2f} km ({elapsed:.2f}s)')\n            success_count += 1\n            \n        except nx.NetworkXNoPath:\n            print('❌ No path found')\n            fail_count += 1\n        except Exception as e:\n            print(f'❌ Error: {str(e)}')\n            fail_count += 1\n\nprint(f'\\n✅ Pre-calculation complete!')\nprint(f'   Success: {success_count}')\nprint(f'   Failed: {fail_count}')\nprint(f'   Total time: {total_time:.2f}s')\nprint(f'   Average per path: {total_time/max(success_count, 1):.2f}s')

In [ ]:
# Save cache to file\noutput_file = Path('collector/outputs/shortest_paths_cache.json')\noutput_file.parent.mkdir(parents=True, exist_ok=True)\n\ncache_data = {\n    'metadata': {\n        'created_at': datetime.now().isoformat(),\n        'total_paths': len(cache),\n        'traffic_points_count': len(points),\n        'computation_time_s': round(total_time, 2)\n    },\n    'paths': cache\n}\n\nwith open(output_file, 'w', encoding='utf-8') as f:\n    json.dump(cache_data, f, indent=2)\n\nfile_size_kb = output_file.stat().st_size / 1024\nprint(f'💾 Saved cache to: {output_file}')\nprint(f'   File size: {file_size_kb:.2f} KB')\nprint(f'   Total paths: {len(cache)}')

In [ ]:
# Display summary statistics\nprint('\\n📊 Cache Statistics:')\nprint(f'   Total cached paths: {len(cache)}')\n\nif cache:\n    distances = [p['distance_km'] for p in cache.values()]\n    print(f'   Shortest route: {min(distances):.2f} km')\n    print(f'   Longest route: {max(distances):.2f} km')\n    print(f'   Average distance: {sum(distances)/len(distances):.2f} km')\n\nprint('\\n✅ Done! You can now use this cache in your server.')